In [16]:
# !rm -rf /content/anti_echo

In [1]:
!rm -rf /content/sample_data/

Setup 1 of N - environment and workspace

Purpose
Install core dependencies, set up a clean workspace at /content/anti_echo, and print diagnostics so collaborators can debug quickly.

Why this matters
Pinned installs reduce breakage. A consistent folder layout keeps artifacts predictable. Diagnostics help when the runtime changes.

Outputs
Directories created under /content/anti_echo, environment flags set, package versions printed.

In [2]:
# Setup 1 of N: environment and workspace
# Colab safe. No Drive mount. Heavy comments for clarity.

import os
import sys
import subprocess
import textwrap
from pathlib import Path

def pip_install(pkgs):
    # Quiet installs but still show what is being installed for reproducibility
    cmd = [sys.executable, "-m", "pip", "install", "-q"] + pkgs
    print("Installing:", " ".join(pkgs))
    subprocess.check_call(cmd)

# Core dependencies with sane pins or upper bounds
pip_install([
    "feedparser==6.0.10",                    # RSS parsing
    "trafilatura>=1.6.2,<2.0",               # robust article extraction
    "sentence-transformers>=2.6.1,<3.0",     # embeddings
    "chromadb>=0.5.5,<0.6.0",                # local vector store
    "huggingface_hub>=0.24.0,<0.28.0",       # HF dataset and file ops
    "pyyaml>=6.0.1,<7.0",                    # config parsing
    "numpy>=1.26.4,<3.0",                    # arrays
    "tqdm>=4.66.0,<5.0",                     # progress
    "requests>=2.31.0,<3.0",                 # HTTP
    "rapidfuzz>=3.6.0,<4.0",                 # dedupe or fuzzy utils
    "scikit-learn>=1.4.0,<2.0",              # clustering for topics
    "transformers>=4.43.0,<5.0",             # summarization model
    "nltk>=3.8.1,<4.0"                       # sentence splitting
])

# Optional accelerator. If import fails, install a compatible torch
try:
    import torch
except Exception:
    pip_install(["torch>=2.2.0,<3.0"])
    import torch

# Define a single project root for all artifacts in this session
PROJECT_ROOT = Path("/content/anti_echo").resolve()
SUBDIRS = [
    "raw",        # scraped article text and sidecar metadata
    "batches",    # packaged embeddings and manifest before HF upload
    "chroma_db",  # local persistent Chroma store
    "logs",       # run logs
    "feeds",      # index and feed state
    "tmp"         # scratch space
]
for d in SUBDIRS:
    (PROJECT_ROOT / d).mkdir(parents=True, exist_ok=True)

# Set environment flags to reduce noise and avoid accidental multithreading bugs
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CHROMA_TELEMETRY_ENABLED"] = "false"
os.environ["ANONYMIZED_TELEMETRY"] = "false"

# Print environment diagnostics for reproducibility and easy debugging
import platform, json
from importlib.metadata import version, PackageNotFoundError

def v(name):
    try:
        return version(name)
    except PackageNotFoundError:
        return "not-installed"

info = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "packages": {
        "feedparser": v("feedparser"),
        "trafilatura": v("trafilatura"),
        "sentence-transformers": v("sentence-transformers"),
        "chromadb": v("chromadb"),
        "huggingface_hub": v("huggingface-hub"),
        "PyYAML": v("PyYAML"),
        "numpy": v("numpy"),
        "rapidfuzz": v("rapidfuzz"),
        "torch": v("torch"),
        "tqdm": v("tqdm"),
        "requests": v("requests"),
        "scikit-learn": v("scikit-learn"),
        "transformers": v("transformers"),
        "nltk": v("nltk"),
    },
    "paths": {
        "project_root": str(PROJECT_ROOT),
        "raw": str(PROJECT_ROOT / "raw"),
        "batches": str(PROJECT_ROOT / "batches"),
        "chroma_db": str(PROJECT_ROOT / "chroma_db"),
        "logs": str(PROJECT_ROOT / "logs"),
        "feeds": str(PROJECT_ROOT / "feeds"),
        "tmp": str(PROJECT_ROOT / "tmp"),
    }
}

# CUDA info is helpful to know if summarization can use GPU
info["cuda_available"] = bool(torch.cuda.is_available())
if info["cuda_available"]:
    info["cuda_device_name"] = torch.cuda.get_device_name(0)

print(json.dumps(info, indent=2))

# A tiny workspace readme helps collaborators quickly orient
workspace_readme = PROJECT_ROOT / "README_WORKSPACE.txt"
if not workspace_readme.exists():
    workspace_readme.write_text(textwrap.dedent("""
        anti echo chamber - Colab workspace
        This directory is ephemeral per session.
        Do not commit files from here directly.
        Subdirs:
          raw        - local scraped texts and meta for this session
          batches    - locally packaged batches before HF upload
          chroma_db  - local Chroma rebuild target
          logs       - run logs
          feeds      - runtime feed artifacts
          tmp        - scratch space
    """).strip() + "\n", encoding="utf-8")
print(f"Workspace ready at {PROJECT_ROOT}")


Installing: feedparser==6.0.10 trafilatura>=1.6.2,<2.0 sentence-transformers>=2.6.1,<3.0 chromadb>=0.5.5,<0.6.0 huggingface_hub>=0.24.0,<0.28.0 pyyaml>=6.0.1,<7.0 numpy>=1.26.4,<3.0 tqdm>=4.66.0,<5.0 requests>=2.31.0,<3.0 rapidfuzz>=3.6.0,<4.0 scikit-learn>=1.4.0,<2.0 transformers>=4.43.0,<5.0 nltk>=3.8.1,<4.0
{
  "python": "3.12.12",
  "platform": "Linux-6.6.105+-x86_64-with-glibc2.35",
  "packages": {
    "feedparser": "6.0.10",
    "trafilatura": "1.12.2",
    "sentence-transformers": "2.7.0",
    "chromadb": "0.5.23",
    "huggingface_hub": "0.27.1",
    "PyYAML": "6.0.3",
    "numpy": "2.0.2",
    "rapidfuzz": "3.14.1",
    "torch": "2.8.0+cu126",
    "tqdm": "4.67.1",
    "requests": "2.32.4",
    "scikit-learn": "1.6.1",
    "transformers": "4.46.3",
    "nltk": "3.9.1"
  },
  "paths": {
    "project_root": "/content/anti_echo",
    "raw": "/content/anti_echo/raw",
    "batches": "/content/anti_echo/batches",
    "chroma_db": "/content/anti_echo/chroma_db",
    "logs": "/content

Setup 2 of N - config and paths bootstrap

Purpose
Pull config and label maps from your GitHub repo, cache them locally in the Colab session, validate required keys, and create runtime dirs from config.

Why this matters
Single source of truth for model names, dims, batch filenames, and collection names. Keeps the notebook aligned with the repo.

Outputs
CONFIG, STANCE_AXES, TOPIC_LABELS in memory, directories created, and HF_DATASET_ID exported to env.


In [3]:
# ==============================
# Setup 2 of N: config and paths bootstrap (v4.1)
# Loads config.yaml from GitHub, fetches topics.json,
# and ensures ideological JSONs (political_leanings & implied_stances) are cached locally.
# ==============================

import os
import json
import yaml
import requests
from pathlib import Path

PROJECT_ROOT = Path("/content/anti_echo").resolve()
CONFIG_CACHE = PROJECT_ROOT / "config_cache"
CONFIG_DIR = PROJECT_ROOT / "config"
CONFIG_CACHE.mkdir(parents=True, exist_ok=True)
CONFIG_DIR.mkdir(parents=True, exist_ok=True)

# --- GitHub repo location ---
REPO_OWNER = "AHMerrill"
REPO_NAME = "anti-echo-chamber"
BRANCH = "main"

def raw_url(path: str) -> str:
    """Compose a raw GitHub URL for a given path in the repo."""
    return f"https://raw.githubusercontent.com/{REPO_OWNER}/{REPO_NAME}/{BRANCH}/{path.lstrip('/')}"

def fetch_text_first(paths):
    """Try multiple candidate filenames and return the first successful fetch."""
    last_err = None
    for p in paths:
        url = raw_url(p)
        try:
            r = requests.get(url, timeout=20)
            if r.status_code == 200 and r.text.strip():
                return r.text, p, url
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Could not fetch any of {paths}. Last error: {last_err}")

# --- Candidate config filenames ---
CFG_CANDIDATES = ["config/config.yaml", "config/config.yml", "config/config.json"]

# --- Fetch and cache main config ---
cfg_txt, cfg_path, cfg_url = fetch_text_first(CFG_CANDIDATES)
(CONFIG_CACHE / Path(cfg_path).name).write_text(cfg_txt, encoding="utf-8")

# --- Parse config ---
CONFIG = yaml.safe_load(cfg_txt) if cfg_path.endswith((".yaml", ".yml")) else json.loads(cfg_txt)

# --- Fetch topics.json directly from GitHub ---
TOPICS_PATH = CONFIG_DIR / "topics.json"
TOPICS_URL = raw_url("config/topics.json")

if not TOPICS_PATH.exists():
    print(f"Fetching topics.json from {TOPICS_URL} ...")
    r = requests.get(TOPICS_URL, timeout=20)
    r.raise_for_status()
    TOPICS_PATH.write_text(r.text, encoding="utf-8")
    print(f"Saved topics.json to {TOPICS_PATH}")

try:
    TOPIC_LABELS = json.load(open(TOPICS_PATH, encoding="utf-8"))
    print(f"Loaded {len(TOPIC_LABELS)} topic clusters from topics.json")
except Exception as e:
    print(f"Warning: Failed to load topics.json ({e})")
    TOPIC_LABELS = {}

# --- Ensure political_leanings.json and implied_stances.json are cached locally ---
def fetch_if_missing(filename, repo_subpath):
    local_path = CONFIG_DIR / filename
    if not local_path.exists():
        url = raw_url(f"config/{repo_subpath}")
        print(f"Fetching {filename} from {url} ...")
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        local_path.write_text(r.text, encoding="utf-8")
        print(f"Saved {filename} to {local_path}")
    return local_path

LEANINGS_PATH = fetch_if_missing("political_leanings.json", "political_leanings.json")
STANCES_PATH  = fetch_if_missing("implied_stances.json", "implied_stances.json")

POLITICAL_LEANINGS = json.load(open(LEANINGS_PATH, encoding="utf-8"))
IMPLIED_STANCES    = json.load(open(STANCES_PATH, encoding="utf-8"))

# --- Mirror config.yaml into /config for downstream setups ---
CONFIG_PATH = CONFIG_DIR / "config.yaml"
if not CONFIG_PATH.exists():
    src_path = CONFIG_CACHE / Path(cfg_path).name
    CONFIG_PATH.write_text(open(src_path, "r", encoding="utf-8").read(), encoding="utf-8")
    print(f"Copied config.yaml from cache to {CONFIG_PATH}")

# --- Validate essential keys ---
required_cfg_keys = ["hf_dataset_id", "chroma_collections", "embeddings", "batch", "ids", "chroma"]
missing = [k for k in required_cfg_keys if k not in CONFIG]
if missing:
    raise ValueError(f"Missing required config keys: {missing}")

# --- Create required directories ---
(Path(PROJECT_ROOT / CONFIG["batch"]["base_dir"])).mkdir(parents=True, exist_ok=True)
(Path(PROJECT_ROOT / CONFIG["chroma"]["dir"])).mkdir(parents=True, exist_ok=True)
(Path(PROJECT_ROOT / CONFIG.get("logging", {}).get("save_dir", "logs"))).mkdir(parents=True, exist_ok=True)

# --- Print runtime summary ---
print(json.dumps({
    "hf_dataset_id": CONFIG["hf_dataset_id"],
    "collections": CONFIG["chroma_collections"],
    "embeddings": CONFIG["embeddings"],
    "topics": {
        "count": len(TOPIC_LABELS),
        "source": str(TOPICS_PATH)
    },
    "ideology_files": {
        "political_leanings": str(LEANINGS_PATH),
        "implied_stances": str(STANCES_PATH)
    },
    "paths": {
        "batches": str(PROJECT_ROOT / CONFIG["batch"]["base_dir"]),
        "chroma_db": str(PROJECT_ROOT / CONFIG["chroma"]["dir"]),
        "config_yaml": str(CONFIG_PATH)
    },
    "config_source": cfg_url
}, indent=2))

# --- Environment variable for downstream setups ---
os.environ["HF_DATASET_ID"] = CONFIG["hf_dataset_id"]


Fetching topics.json from https://raw.githubusercontent.com/AHMerrill/anti-echo-chamber/main/config/topics.json ...
Saved topics.json to /content/anti_echo/config/topics.json
Loaded 1400 topic clusters from topics.json
Fetching political_leanings.json from https://raw.githubusercontent.com/AHMerrill/anti-echo-chamber/main/config/political_leanings.json ...
Saved political_leanings.json to /content/anti_echo/config/political_leanings.json
Fetching implied_stances.json from https://raw.githubusercontent.com/AHMerrill/anti-echo-chamber/main/config/implied_stances.json ...
Saved implied_stances.json to /content/anti_echo/config/implied_stances.json
Copied config.yaml from cache to /content/anti_echo/config/config.yaml
{
  "hf_dataset_id": "zanimal/anti-echo-artifacts",
  "collections": {
    "topic": "news_topic",
    "stance": "news_stance"
  },
  "embeddings": {
    "topic_model": "intfloat/e5-base-v2",
    "stance_model": "Snowflake/snowflake-arctic-embed-l",
    "dim": 1024,
    "dtype

Setup 3 of N - authentication for Hugging Face and GitHub

Purpose
Load tokens into environment and verify access. Later cells use these tokens to push to HF and update your GitHub registry.

Why this matters
Catching auth issues early prevents failing halfway through a run.

Outputs
Logged in to HF, GitHub token validated.

In [4]:
# Setup 3 of N: auth for Hugging Face and GitHub
# Prompts only if tokens are not already in the environment.

import os
import requests
from getpass import getpass
from huggingface_hub import login, whoami

def need(envvar, prompt):
    # Request once per session if missing
    if not os.environ.get(envvar, "").strip():
        os.environ[envvar] = getpass(prompt)
    print(f"{envvar} loaded:", bool(os.environ.get(envvar)))

# Gather tokens
need("HF_TOKEN", "Enter your Hugging Face token: ")
need("GITHUB_TOKEN", "Enter your GitHub Personal Access Token: ")

# Sign in to HF so upload_file works later
try:
    login(token=os.environ["HF_TOKEN"], add_to_git_credential=False)
    print("HF login OK:", whoami(token=os.environ["HF_TOKEN"]).get("name","(ok)"))
except Exception as e:
    print("HF login failed:", e)

# Quick GitHub check to confirm token scopes
try:
    r = requests.get(
        "https://api.github.com/user",
        headers={"Authorization": f"Bearer {os.environ['GITHUB_TOKEN']}"},
        timeout=15
    )
    print("GitHub auth status:", r.status_code)
except Exception as e:
    print("GitHub auth check failed:", e)


Enter your Hugging Face token: ··········
HF_TOKEN loaded: True
Enter your GitHub Personal Access Token: ··········
GITHUB_TOKEN loaded: True


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


HF login OK: zanimal
GitHub auth status: 200


Setup 4 of N - restore feed index and feed state

Purpose
Restore feeds/index.json and feeds/feeds_state.json from HF latest pointers, fallback to GitHub if missing, or reconstruct from HF batch metadata if neither exists.

Why this matters
Prevents re scraping the same URLs, and keeps numbering and batching consistent across runs.

Outputs
feeds/index.json and feeds/feeds_state.json present locally with a quick summary.

In [5]:
# Setup 4 of N: restore feed index and state
# Guarantees local index and state exist before scraping.

import os, json, shutil, requests, datetime as dt, re, hashlib
from pathlib import Path
from huggingface_hub import hf_hub_download

PROJECT_ROOT = Path("/content/anti_echo").resolve()
FEEDS_DIR = PROJECT_ROOT / "feeds"
FEEDS_DIR.mkdir(parents=True, exist_ok=True)

INDEX_PATH = FEEDS_DIR / "index.json"
STATE_PATH = FEEDS_DIR / "feeds_state.json"
HF_DATASET_ID = os.environ["HF_DATASET_ID"]

REPO_OWNER = "AHMerrill"
REPO_NAME = "anti-echo-chamber"
BRANCH = "main"

def try_hf_restore() -> bool:
    # Prefer HF because it is the single source of truth in this design
    try:
        st = hf_hub_download(HF_DATASET_ID, "feeds/feeds_state_latest.json", repo_type="dataset")
        ix = hf_hub_download(HF_DATASET_ID, "feeds/feed_index_latest.json", repo_type="dataset")
        shutil.copy(st, STATE_PATH)
        shutil.copy(ix, INDEX_PATH)
        print("Restored feed state from HF latest")
        return True
    except Exception as e:
        print("HF latest not found:", e)
        return False

def try_github_restore() -> bool:
    # Fallback if HF latest pointers are not present yet
    try:
        base = f"https://raw.githubusercontent.com/{REPO_OWNER}/{REPO_NAME}/{BRANCH}/feeds"
        got = False
        for src, dst in [("feeds_state_latest.json", STATE_PATH), ("feed_index_latest.json", INDEX_PATH)]:
            r = requests.get(f"{base}/{src}", timeout=20)
            if r.status_code == 200 and r.text.strip():
                dst.write_text(r.text, encoding="utf-8")
                got = True
        if got:
            print("Restored feed state from GitHub latest")
        return got
    except Exception as e:
        print("GitHub restore failed:", e)
        return False

restored = try_hf_restore() or try_github_restore()

if not restored:
    # Reconstruct from HF batch metadata listed in artifacts_registry.json
    print("No latest feed state found. Attempting reconstruction from HF batches...")
    REGISTRY_URL = f"https://raw.githubusercontent.com/{REPO_OWNER}/{REPO_NAME}/{BRANCH}/artifacts/artifacts_registry.json"
    REGISTRY = requests.get(REGISTRY_URL, timeout=20).json()

    # Collect metadata jsonl from each batch
    metas = []
    for b in REGISTRY.get("batches", []):
        meta_rel = (b.get("hf_paths") or b.get("paths") or {}).get("metadata")
        if not meta_rel:
            continue
        try:
            metas.append(Path(hf_hub_download(HF_DATASET_ID, meta_rel, repo_type="dataset")))
        except Exception as e:
            print("Skip meta fetch:", e)

    # Build a minimal index of known URLs to prevent re scraping
    items = {}
    def norm(txt): return re.sub(r"\s+"," ", txt.strip().lower())
    def sha256_text(txt): return hashlib.sha256(norm(txt).encode()).hexdigest()

    for fp in metas:
        for line in fp.read_text(encoding="utf-8").splitlines():
            if not line.strip():
                continue
            try:
                obj = json.loads(line)
            except Exception:
                continue
            u = obj.get("url")
            if u and u not in items:
                items[u] = {"status": "ok", "fetched_at": dt.datetime.now(dt.timezone.utc).isoformat()}

    INDEX_PATH.write_text(json.dumps({"last_updated": dt.datetime.now(dt.timezone.utc).isoformat(),"items": items}, indent=2), encoding="utf-8")

    # Create a simple ring buffer structure for each feed
    url_hashes = [sha256_text(u)[:12] for u in items.keys()]
    feeds_block = {
        "commentisfree": {"feed_url": None, "recent_url_hashes": url_hashes[-1000:], "recent_url_hashes_max": 1000},
        "theguardian": {"feed_url": None, "recent_url_hashes": url_hashes[-500:], "recent_url_hashes_max": 500},
    }
    STATE_PATH.write_text(json.dumps({"version":1,"updated_at": dt.datetime.now(dt.timezone.utc).isoformat(),"feeds": feeds_block}, indent=2), encoding="utf-8")
    print("Reconstruction complete")

# Echo a small summary so you can verify
ix = json.loads(INDEX_PATH.read_text(encoding="utf-8"))
st = json.loads(STATE_PATH.read_text(encoding="utf-8"))
print({"index_items": len(ix.get("items",{})), "feeds": list(st.get("feeds",{}).keys())})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HF latest not found: 404 Client Error. (Request ID: Root=1-68ee9463-69f1831741797da83fa145b3;0848d275-1b49-41cd-8c5a-a617abf1cbf9)

Entry Not Found for url: https://huggingface.co/datasets/zanimal/anti-echo-artifacts/resolve/main/feeds/feeds_state_latest.json.
No latest feed state found. Attempting reconstruction from HF batches...
Reconstruction complete
{'index_items': 0, 'feeds': ['commentisfree', 'theguardian']}


Setup 5 of N - rebuild Chroma from HF batches

Purpose
Create or refresh local Chroma collections from the HF dataset using artifacts/artifacts_registry.json as the ledger. HF is the source of truth. Local Chroma is a cache.

Why this matters
Ensures your retrieval state is consistent before adding new data. No duplicated rows. Clean numbering follows registry order.

Outputs
Two Chroma collections present with counts: topic and stance.

In [6]:
# Setup 5 of N: rebuild Chroma from HF (schema aware)

import os, json, numpy as np, logging, requests
from pathlib import Path
from huggingface_hub import hf_hub_download
import chromadb

# Optional: quiet Chroma telemetry noise in Colab logs
os.environ["CHROMA_TELEMETRY_ENABLED"] = "false"
logging.getLogger("chromadb.telemetry").setLevel(logging.ERROR)

PROJECT_ROOT = Path("/content/anti_echo").resolve()
CHROMA_DIR = PROJECT_ROOT / CONFIG["chroma"]["dir"]
HF_DATASET_ID = CONFIG["hf_dataset_id"]
COLL_TOPIC = CONFIG["chroma_collections"]["topic"]
COLL_STANCE = CONFIG["chroma_collections"]["stance"]
EMB_DIM = int(CONFIG["embeddings"]["dim"])

REPO_OWNER = "AHMerrill"
REPO_NAME = "anti-echo-chamber"
BRANCH = "main"
REGISTRY_URL = f"https://raw.githubusercontent.com/{REPO_OWNER}/{REPO_NAME}/{BRANCH}/artifacts/artifacts_registry.json"

def ensure_chroma():
    client = chromadb.PersistentClient(path=str(CHROMA_DIR))
    t = client.get_or_create_collection(name=COLL_TOPIC, metadata={"hnsw:space": "cosine"})
    s = client.get_or_create_collection(name=COLL_STANCE, metadata={"hnsw:space": "cosine"})
    return client, t, s

def load_npz(fp: Path, dim: int):
    arr = np.load(fp, allow_pickle=False)
    if isinstance(arr, np.lib.npyio.NpzFile):
        arr = arr[list(arr.files)[0]]
    vecs = np.asarray(arr)
    if vecs.ndim != 2 or vecs.shape[1] != dim:
        raise ValueError(f"Bad embedding shape in {fp.name}. Got {vecs.shape}, expected [N,{dim}]")
    if not np.isfinite(vecs).all():
        raise ValueError(f"Non finite values in {fp.name}")
    return vecs

def upsert_chunks(coll, ids, vecs, metas, chunk=2048):
    n = len(ids)
    for i in range(0, n, chunk):
        j = min(i + chunk, n)
        coll.upsert(ids=ids[i:j], embeddings=vecs[i:j].tolist(), metadatas=metas[i:j])

def read_jsonl(fp: Path):
    with fp.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                yield json.loads(line)

# Pull the registry ledger from GitHub
REGISTRY = requests.get(REGISTRY_URL, timeout=20).json()

client, topic_coll, stance_coll = ensure_chroma()
added_rows = 0
ingested_batches = 0
skipped_legacy = 0

for b in REGISTRY.get("batches", []):
    paths = b.get("paths") or b.get("hf_paths") or {}
    sv = b.get("schema_version", 1)

    if sv == 1:
        # Legacy single metadata schema does not align stance rows to topic rows
        print(f"Skip legacy batch {b.get('batch_id')} schema_version=1")
        skipped_legacy += 1
        continue

    need = ["embeddings_topic","embeddings_stance","metadata_topic","metadata_stance"]
    if not all(k in paths for k in need):
        print(f"Skip batch {b.get('batch_id')} missing required paths")
        continue

    # Download artifacts from HF
    t_vecs_path = Path(hf_hub_download(HF_DATASET_ID, paths["embeddings_topic"], repo_type="dataset"))
    s_vecs_path = Path(hf_hub_download(HF_DATASET_ID, paths["embeddings_stance"], repo_type="dataset"))
    t_meta_path = Path(hf_hub_download(HF_DATASET_ID, paths["metadata_topic"], repo_type="dataset"))
    s_meta_path = Path(hf_hub_download(HF_DATASET_ID, paths["metadata_stance"], repo_type="dataset"))

    # Load vectors
    t_vecs = load_npz(t_vecs_path, EMB_DIM)
    s_vecs = load_npz(s_vecs_path, EMB_DIM)

    # Load aligned metadata lists with explicit row ids
    topic_ids, topic_metas = [], []
    for rec in read_jsonl(t_meta_path):
        rid = rec.get("row_id") or f"{rec.get('id','unknown')}::topic::0"
        topic_ids.append(rid)
        topic_metas.append(rec)

    stance_ids, stance_metas = [], []
    for rec in read_jsonl(s_meta_path):
        rid = rec.get("row_id") or f"{rec.get('id','unknown')}::stance::0"
        stance_ids.append(rid)
        stance_metas.append(rec)

    if t_vecs.shape[0] != len(topic_ids):
        raise ValueError(f"Topic row count mismatch in batch {b.get('batch_id')}: vecs {t_vecs.shape[0]} meta {len(topic_ids)}")
    if s_vecs.shape[0] != len(stance_ids):
        raise ValueError(f"Stance row count mismatch in batch {b.get('batch_id')}: vecs {s_vecs.shape[0]} meta {len(stance_ids)}")

    # Upsert both collections
    upsert_chunks(topic_coll, topic_ids, t_vecs, topic_metas)
    upsert_chunks(stance_coll, stance_ids, s_vecs, stance_metas)

    added_rows += len(topic_ids) + len(stance_ids)
    ingested_batches += 1
    print(f"Ingested {b.get('batch_id')} topic {len(topic_ids)} stance {len(stance_ids)}")

print({
    "topic_count": topic_coll.count(),
    "stance_count": stance_coll.count(),
    "rows_added": added_rows,
    "ingested_batches": ingested_batches,
    "skipped_legacy_batches": skipped_legacy
})


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


{'topic_count': 0, 'stance_count': 0, 'rows_added': 0, 'ingested_batches': 0, 'skipped_legacy_batches': 0}


Setup 6 of N - scraping tunables

Purpose:
Define scrape quotas, date floor, and feed list. Export to environment so the scraper reads configuration without edits.

Scope:
Attempted to have an even balance of left, right, center for this experiment.  used allsides.com and chatGPT to help cover a complete spectrum of sites considered biased in particular directions


Why this matters
Allows you to adjust scrape size and distribution per run from a single cell.

Outputs
Environment variables set and a printed summary.

In [7]:
# Setup 6 of N: feed configuration
# Defines the set of RSS/Atom feeds to pull from for balanced, full-text content scraping.

import json, os

# Verified full-text feeds (as of latest probe)
NEWS_FEEDS = [
    ("vox", "https://www.vox.com/rss/index.xml"),
    ("cnn-opinion", "http://rss.cnn.com/rss/cnn_opinion.rss"),
    ("guardian", "https://www.theguardian.com/uk/rss"),
    ("bbc", "https://feeds.bbci.co.uk/news/rss.xml"),
    ("cityjournal", "https://www.city-journal.org/rss.xml"),
    ("dailycaller", "https://dailycaller.com/feed/"),
    ("theconversation", "https://theconversation.com/us/articles.atom"),
    ("aljazeera", "https://www.aljazeera.com/xml/rss/all.xml"),
    ("foxnews", "https://moxie.foxnews.com/google-publisher/latest.xml"),
    ("npr", "https://feeds.npr.org/1001/rss.xml"),
    ("reason", "https://reason.com/feed/"),
    ("thefederalist", "https://thefederalist.com/feed/"),
    ("msnbc", "https://feeds.nbcnews.com/msnbc/public/news"),
    ("nypost", "https://nypost.com/feed/"),
]

# Export to environment for scraper
os.environ["NEWS_FEEDS_JSON"] = json.dumps(NEWS_FEEDS)

# Scraper configuration
# Adjust MAX_ARTICLES and MAX_PER_FEED for your target scale
os.environ["MAX_ARTICLES"] = "2800"       # ≈200 per feed × 14 feeds
os.environ["MAX_PER_FEED"] = "200"
os.environ["EVEN_SPLIT"] = "true"
os.environ["DATE_FROM"] = "2019-01-01"    # scrape articles newer than this date
os.environ["FORCE_REFETCH"] = "false"
os.environ["QUOTA_REMAINDER_TO"] = "theconversation"

print("Configured verified full-text feeds:")
for name, url in NEWS_FEEDS:
    print(f"- {name}: {url}")

print("\nScraper parameters:")
print(json.dumps({
    "MAX_ARTICLES": os.environ["MAX_ARTICLES"],
    "MAX_PER_FEED": os.environ["MAX_PER_FEED"],
    "DATE_FROM": os.environ["DATE_FROM"],
    "EVEN_SPLIT": os.environ["EVEN_SPLIT"],
    "FORCE_REFETCH": os.environ["FORCE_REFETCH"],
    "QUOTA_REMAINDER_TO": os.environ["QUOTA_REMAINDER_TO"]
}, indent=2))


Configured verified full-text feeds:
- vox: https://www.vox.com/rss/index.xml
- cnn-opinion: http://rss.cnn.com/rss/cnn_opinion.rss
- guardian: https://www.theguardian.com/uk/rss
- bbc: https://feeds.bbci.co.uk/news/rss.xml
- cityjournal: https://www.city-journal.org/rss.xml
- dailycaller: https://dailycaller.com/feed/
- theconversation: https://theconversation.com/us/articles.atom
- aljazeera: https://www.aljazeera.com/xml/rss/all.xml
- foxnews: https://moxie.foxnews.com/google-publisher/latest.xml
- npr: https://feeds.npr.org/1001/rss.xml
- reason: https://reason.com/feed/
- thefederalist: https://thefederalist.com/feed/
- msnbc: https://feeds.nbcnews.com/msnbc/public/news
- nypost: https://nypost.com/feed/

Scraper parameters:
{
  "MAX_ARTICLES": "2800",
  "MAX_PER_FEED": "200",
  "DATE_FROM": "2019-01-01",
  "EVEN_SPLIT": "true",
  "FORCE_REFETCH": "false",
  "QUOTA_REMAINDER_TO": "theconversation"
}


Setup 7 of N - scraper with dedupe

Purpose
Scrape the feeds, skip URLs already in feeds/index.json, save raw/{id}.txt and {id}.meta.json, and update both feeds/index.json and feeds/feeds_state.json.

Why this matters
Prevents duplicates, keeps state consistent across runs, and prepares clean inputs for embedding.

Outputs
New articles saved under raw/, updated feeds/index.json and feeds/feeds_state.json, and a summary.

In [8]:
# Setup 7 of N: scraper with dedupe and caching
# Uses feed list from Setup 6 (NEWS_FEEDS_JSON)
# Fetches full text locally; saves metadata for embeddings downstream.

import os, re, json, hashlib, datetime as dt
from pathlib import Path
from urllib.parse import urlparse
from email.utils import parsedate_to_datetime
import feedparser, trafilatura, requests

feedparser.USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"

def safe_parse_feed(url):
    """Fetch RSS/Atom with realistic browser headers to bypass blocks."""
    try:
        headers = {
            "User-Agent": feedparser.USER_AGENT,
            "Accept": "application/rss+xml, application/atom+xml, application/xml;q=0.9,*/*;q=0.8",
            "Referer": "https://www.google.com/",
            "Accept-Language": "en-US,en;q=0.9",
        }
        resp = requests.get(url, headers=headers, timeout=15)
        if resp.status_code != 200:
            print(f"feed fetch error ({resp.status_code}): {url}")
            return feedparser.FeedParserDict(entries=[])
        return feedparser.parse(resp.content)
    except Exception as e:
        print(f"feed fetch exception for {url}: {e}")
        return feedparser.FeedParserDict(entries=[])

# === Paths ===
PROJECT_ROOT = Path("/content/anti_echo").resolve()
RAW_DIR = PROJECT_ROOT / "raw"
FEEDS_DIR = PROJECT_ROOT / "feeds"
FEEDS_DIR.mkdir(parents=True, exist_ok=True)
INDEX_PATH = FEEDS_DIR / "index.json"
STATE_PATH = FEEDS_DIR / "feeds_state.json"

# === Load config ===
FEED_LIST = json.loads(os.environ["NEWS_FEEDS_JSON"])
MAX_ARTICLES = int(os.environ.get("MAX_ARTICLES", "30"))
MAX_PER_FEED = os.environ.get("MAX_PER_FEED", "")
MAX_PER_FEED = None if MAX_PER_FEED == "" else int(MAX_PER_FEED)
DATE_FROM = os.environ.get("DATE_FROM", "") or None
FORCE_REFETCH = os.environ.get("FORCE_REFETCH", "false").lower() == "true"
EVEN_SPLIT = os.environ.get("EVEN_SPLIT", "true").lower() == "true"
QUOTA_REMAINDER_TO = os.environ.get("QUOTA_REMAINDER_TO", "theconversation")

def now_iso(): return dt.datetime.now(dt.timezone.utc).isoformat()

# === State & index handling ===
def load_json(path, default):
    if path.exists():
        try: return json.loads(path.read_text(encoding="utf-8"))
        except Exception: pass
    return default

index = load_json(INDEX_PATH, {"last_updated": None, "items": {}})
feeds_state = load_json(STATE_PATH, {"version": 1, "updated_at": None, "feeds": {}})
fs = feeds_state.setdefault("feeds", {})
for name, feed_url in FEED_LIST:
    fs.setdefault(name, {"feed_url": feed_url, "recent_url_hashes": [], "recent_url_hashes_max": 1000, "last_run_at": None})

def save_json(path, obj):
    obj["last_updated"] = now_iso()
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")

# === Utility ===
def parse_date(entry):
    for k in ["published", "updated"]:
        val = getattr(entry, k, None) or entry.get(k)
        if val:
            try: return parsedate_to_datetime(val)
            except Exception: pass
    return None

def in_range(d, lower_iso):
    if not lower_iso: return True
    try: floor = dt.datetime.fromisoformat(lower_iso).replace(tzinfo=dt.timezone.utc)
    except Exception: return True
    if d is None: return True
    if d.tzinfo is None: d = d.replace(tzinfo=dt.timezone.utc)
    return d >= floor

def normalize_text(txt): return re.sub(r"\s+", " ", txt.strip().lower())
def sha256_text(txt): return hashlib.sha256(txt.encode()).hexdigest()
def slugify(text, maxlen=60):
    s = re.sub(r"[^a-zA-Z0-9]+", "-", text).strip("-").lower()
    return s[:maxlen] or "untitled"

def fetch_article(url):
    html = trafilatura.fetch_url(url, no_ssl=False)
    if not html:
        raise RuntimeError("fetch failed")
    text = trafilatura.extract(html, include_comments=False, include_tables=False) or ""
    title_match = re.search(r"<title>(.*?)</title>", html or "", flags=re.I | re.S)
    title = re.sub(r"\s+", " ", title_match.group(1)).strip() if title_match else "Untitled"
    if not text.strip():
        raise RuntimeError("extraction empty")
    return title, text

def already_cached(url):
    return url in index["items"] and index["items"][url].get("status") == "ok"

def mark(url, status):
    index["items"][url] = {"status": status, "fetched_at": now_iso()}
    save_json(INDEX_PATH, index)

# === Quotas ===
feed_names = [n for n, _ in FEED_LIST]
if EVEN_SPLIT:
    base = MAX_ARTICLES // len(feed_names)
    rem = MAX_ARTICLES % len(feed_names)
    quotas = {n: base for n in feed_names}
    quotas[QUOTA_REMAINDER_TO if QUOTA_REMAINDER_TO in quotas else feed_names[0]] += rem
else:
    quotas = {n: 0 for n in feed_names}
    quotas[QUOTA_REMAINDER_TO if QUOTA_REMAINDER_TO in quotas else feed_names[0]] = MAX_ARTICLES
if isinstance(MAX_PER_FEED, int):
    quotas = {k: min(v, MAX_PER_FEED) for k, v in quotas.items()}
print("Quotas:", quotas)

# === Main fetch loop ===
saved_global = 0
errors_global = 0
seen_global = set()

for name, feed_url in FEED_LIST:
    if saved_global >= MAX_ARTICLES: break
    quota = quotas.get(name, 0)
    if quota <= 0: continue

    fp = safe_parse_feed(feed_url)
    if not fp.entries:
        print(f"[{name}] no entries found.")
        continue

    items = []
    for e in fp.entries:
        url = getattr(e, "link", None)
        if not url: continue
        pub = parse_date(e)
        if in_range(pub, DATE_FROM):
            items.append({"url": url, "published": pub})

    uniq, seen = [], set()
    for it in sorted(items, key=lambda x: (x["published"] or dt.datetime.min), reverse=True):
        if it["url"] in seen: continue
        seen.add(it["url"])
        uniq.append(it)

    saved_this = 0
    for it in uniq:
        if saved_global >= MAX_ARTICLES or saved_this >= quota: break
        url = it["url"]
        if url in seen_global: continue
        seen_global.add(url)
        if already_cached(url) and not FORCE_REFETCH:
            print(f"skip (cached) [{name}]: {url}")
            continue
        try:
            title, text = fetch_article(url)
            domain = urlparse(url).netloc
            slug = slugify(title)
            h = sha256_text(normalize_text(text))
            art_id = f"{domain}-{slug}-{h[:12]}"
            RAW_DIR.mkdir(parents=True, exist_ok=True)
            txt_path = RAW_DIR / f"{art_id}.txt"
            meta_path = RAW_DIR / f"{art_id}.meta.json"
            txt_path.write_text(text, encoding="utf-8")
            meta = {
                "id": art_id, "url": url, "title": title, "source": name, "domain": domain,
                "published": it["published"].isoformat() if it["published"] else None,
                "sha256": h, "chars": len(text), "saved_at": now_iso(),
            }
            meta_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")
            mark(url, "ok")
            fs[name]["last_run_at"] = now_iso()
            saved_this += 1
            saved_global += 1
            print(f"saved [{name}]: {txt_path.name} | {title[:90]} | {len(text)} chars")
        except Exception as e:
            mark(url, "error")
            errors_global += 1
            print(f"error [{name}]: {url} | {type(e).__name__}: {str(e)[:120]}")

feeds_state["updated_at"] = now_iso()
STATE_PATH.write_text(json.dumps(feeds_state, indent=2), encoding="utf-8")

print(json.dumps({
    "saved_total": saved_global,
    "errors_total": errors_global,
    "index_items": len(index["items"]),
    "feeds_state_path": str(STATE_PATH)
}, indent=2))


Quotas: {'vox': 200, 'cnn-opinion': 200, 'guardian': 200, 'bbc': 200, 'cityjournal': 200, 'dailycaller': 200, 'theconversation': 200, 'aljazeera': 200, 'foxnews': 200, 'npr': 200, 'reason': 200, 'thefederalist': 200, 'msnbc': 200, 'nypost': 200}
saved [vox]: www.vox.com-is-it-ok-for-my-kids-to-watch-youtube-an-expert-explains-vox-0c9bd26d2384.txt | Is it OK for my kids to watch YouTube? An expert explains. | Vox | 5303 chars
saved [vox]: www.vox.com-how-lenacapavir-can-change-the-fight-against-hiv-if-we-can-l-622170eb6794.txt | How lenacapavir can change the fight against HIV — if we can let it | Vox | 12169 chars
saved [vox]: www.vox.com-can-american-democracy-recover-from-trump-here-s-what-new-da-b2beba71c4a1.txt | Can American democracy recover from Trump? Here’s what new data says. | Vox | 9393 chars
saved [vox]: www.vox.com-the-uncomfortable-problem-with-america-s-greatest-civil-righ-046eb1e018d5.txt | The uncomfortable problem with America’s greatest civil rights law, the Voting 

Setup 9 of N - topic embeddings to Chroma

Purpose
Generate multi topic embeddings per article and upsert into the topic collection. This models what the article is about. Each article may get multiple topic vectors.

Why this matters
Topical neighbors power the first half of contrastive retrieval.

Outputs
Vectors upserted into the topic collection with structured metadata.

In [9]:
# ==============================
# Setup 9 of N: topic embeddings (multi-topic mapping with cosine similarity)
# Chroma-safe metadata: primitives only
# ==============================

import os, json, time, numpy as np, nltk, torch, warnings
from pathlib import Path
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import chromadb

warnings.filterwarnings("ignore", message="Token indices sequence length is longer")

PROJECT_ROOT = Path("/content/anti_echo").resolve()
RAW_DIR = PROJECT_ROOT / "raw"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and topic config from CONFIG and Setup 2 globals
topic_model_name = CONFIG["embeddings"]["topic_model"]
topic_dim = int(CONFIG["embeddings"]["dim"])
topic_dtype = CONFIG["embeddings"]["dtype"]
chunk_tokens = int(CONFIG["embeddings"]["chunk_tokens"])
MAX_TOPICS = CONFIG["topics"].get("max_topics_per_article", 5)
TOPIC_THRESHOLD = CONFIG["topics"].get("similarity_threshold", 0.4)

# Ensure NLTK sentence tokenizer is available
for pkg in ["punkt", "punkt_tab"]:
    try:
        nltk.data.find(f"tokenizers/{pkg}")
    except LookupError:
        nltk.download(pkg)

tokenizer = AutoTokenizer.from_pretrained(topic_model_name, use_fast=True)
tokenizer.model_max_length = 512
embedder = SentenceTransformer(topic_model_name, device=device)

# Build topic anchor centroids
print(f"Encoding {len(TOPIC_LABELS)} topic anchors...")
topic_anchors = {}
for label, phrases in TOPIC_LABELS.items():
    embs = embedder.encode(
        phrases,
        normalize_embeddings=True,
        batch_size=8 if device == "cuda" else 16,
        show_progress_bar=False
    )
    topic_anchors[label] = np.mean(np.asarray(embs), axis=0)
print(f"Encoded {len(topic_anchors)} topic centroids.")

# Persistent Chroma connection for topic collection
client = chromadb.PersistentClient(path=str(PROJECT_ROOT / CONFIG["chroma"]["dir"]))
topic_coll = client.get_collection(CONFIG["chroma_collections"]["topic"])

# Utilities
def sent_split(text):
    return [s.strip() for s in nltk.sent_tokenize(text) if s.strip()]

def encode(texts):
    if isinstance(texts, str):
        texts = [texts]
    bs = 4 if torch.cuda.is_available() else 16
    vecs = embedder.encode(
        texts,
        batch_size=bs,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False,
    )
    return np.array(vecs)

def chunk_by_tokens(text, max_tokens=512, overlap=64):
    ids = tokenizer(text, add_special_tokens=False, return_attention_mask=False)["input_ids"]
    step = max_tokens - overlap
    chunks = []
    for i in range(0, len(ids), step):
        j = min(i + max_tokens, len(ids))
        piece = tokenizer.decode(ids[i:j], skip_special_tokens=True)
        if piece.strip():
            chunks.append(piece)
    return chunks

def sanitize(meta: dict):
    out = {}
    for k, v in meta.items():
        if isinstance(v, (str, int, float, bool)) or v is None:
            out[k] = "" if v is None else v
        else:
            out[k] = str(v)
    return out

def topic_vecs(text):
    sents = sent_split(text)
    if not sents:
        return []
    if len(sents) < 2:
        chunks = chunk_by_tokens(" ".join(sents), chunk_tokens, 64)
        vecs = encode(chunks)
        return [vecs.mean(axis=0)]
    emb = encode(sents)
    k = min(max(1, len(sents)//8), 8)
    labels = AgglomerativeClustering(n_clusters=k).fit_predict(emb)
    segs = [" ".join([s for s, l in zip(sents, labels) if l == lab]) for lab in sorted(set(labels))]
    out = []
    for seg in segs:
        chunks = chunk_by_tokens(seg, chunk_tokens, 64)
        if not chunks:
            continue
        pooled = encode(chunks).mean(axis=0)
        out.append(pooled)
    return out

def match_topics(vec):
    scores = {label: cosine_similarity([vec], [anchor])[0][0] for label, anchor in topic_anchors.items()}
    sorted_topics = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    topics = []
    for i, (k, v) in enumerate(sorted_topics[:MAX_TOPICS]):
        if i == 0 or v >= TOPIC_THRESHOLD:
            topics.append({"topic_label": k, "similarity": float(v)})
    if not topics:
        topics = [{"topic_label": "General / Miscellaneous", "similarity": 0.0}]
    return topics

def upsert_in_chunks(collection, ids, vectors, metadatas, chunk=2048):
    n = len(ids)
    for i in range(0, n, chunk):
        j = min(i + chunk, n)
        collection.upsert(
            ids=ids[i:j],
            embeddings=vectors[i:j].tolist(),
            metadatas=metadatas[i:j],
        )

# Main loop
start = time.time()
added = 0
processed = 0

for txt_path in RAW_DIR.glob("*.txt"):
    meta_path = txt_path.with_suffix(".meta.json")
    if not meta_path.exists():
        continue
    text = txt_path.read_text(encoding="utf-8").strip()
    if not text:
        continue
    meta = json.loads(meta_path.read_text(encoding="utf-8"))

    vecs = topic_vecs(text)
    if not vecs:
        continue

    ids, metas = [], []
    for i, v in enumerate(vecs):
        topics_detected = match_topics(v)

        # Prepare Chroma-safe metadata
        topics_json = json.dumps(topics_detected, ensure_ascii=False)
        topics_flat = [t["topic_label"] for t in topics_detected]
        top_topic = topics_detected[0]["topic_label"] if topics_detected else ""

        enriched_meta = sanitize({
            **meta,
            "topic_index": i,
            "topic_model": topic_model_name,
            "topic_labels_json": topics_json,              # JSON string
            "topics_flat": ";".join(topics_flat),         # flat string list for filtering
            "top_topic": top_topic
        })
        ids.append(f"{meta['id']}::topic::{i}")
        metas.append(enriched_meta)

    upsert_in_chunks(topic_coll, ids, np.vstack(vecs), metas)
    added += len(vecs)
    processed += 1

print(f"Processed {processed} articles.")
print(f"Topic upserts: {added} in {round(time.time()-start,2)}s")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Encoding 1400 topic anchors...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors


Encoded 1400 topic centroids.
Processed 434 articles.
Topic upserts: 1675 in 923.68s


In [10]:
# Inspect topic labels saved in Chroma (primitives-only schema)

import os, json, datetime as dt, collections
from pathlib import Path
import pandas as pd
import chromadb

PROJECT_ROOT = Path("/content/anti_echo").resolve()
CHROMA_DIR = PROJECT_ROOT / CONFIG["chroma"]["dir"]
TOPIC_NAME = CONFIG["chroma_collections"]["topic"]

client = chromadb.PersistentClient(path=str(CHROMA_DIR))
topic = client.get_collection(TOPIC_NAME)

n = topic.count()
dump = topic.get(include=["metadatas"], limit=n)

def parse_topics_json(s):
    if not s:
        return []
    try:
        val = json.loads(s)
        return val if isinstance(val, list) else []
    except Exception:
        return []

def labels_to_str(lbls, top_k=5):
    try:
        ordered = sorted(
            [(d.get("topic_label",""), float(d.get("similarity", 0))) for d in lbls if isinstance(d, dict)],
            key=lambda x: x[1],
            reverse=True
        )
    except Exception:
        ordered = [(str(x), None) for x in lbls]
    if top_k:
        ordered = ordered[:top_k]
    return "; ".join([f"{k} ({v:.2f})" if isinstance(v, float) else str(k) for k, v in ordered])

rows = []
for rid, m in zip(dump["ids"], dump["metadatas"]):
    if not isinstance(m, dict):
        continue
    lbls = parse_topics_json(m.get("topic_labels_json", ""))
    rows.append({
        "row_id": rid,
        "id": m.get("id", ""),
        "title": m.get("title", ""),
        "domain": m.get("domain", ""),
        "published": m.get("published", ""),
        "topic_index": m.get("topic_index", None),
        "top_topic": m.get("top_topic", ""),
        "topics_flat": m.get("topics_flat", ""),
        "topics_pretty": labels_to_str(lbls, top_k=5),
        "topic_labels_list": lbls,
    })

df_vec = pd.DataFrame(rows)
if not df_vec.empty:
    df_vec["published_dt"] = pd.to_datetime(df_vec["published"], errors="coerce")
    df_vec = df_vec.sort_values(by=["published_dt","domain","title","topic_index"], ascending=[False, True, True, True]).reset_index(drop=True)

print(f"Per vector rows: {len(df_vec)}")
display(df_vec[["title","domain","published","topic_index","top_topic","topics_pretty"]].fillna(""))

# Aggregate per article by averaging similarity per label
def aggregate_article_topics(group):
    acc = collections.defaultdict(list)
    for lbls in group["topic_labels_list"]:
        for d in lbls:
            if isinstance(d, dict) and "topic_label" in d:
                acc[d["topic_label"]].append(float(d.get("similarity", 0)))
    scored = sorted([(k, sum(v)/max(len(v),1)) for k, v in acc.items()], key=lambda x: x[1], reverse=True)
    top = scored[:5]
    return pd.Series({
        "topics_merged": "; ".join([f"{k} ({v:.2f})" for k, v in top]),
        "topics_json": json.dumps([{ "topic_label": k, "mean_similarity": v } for k, v in top], ensure_ascii=False)
    })

if not df_vec.empty:
    group_cols = ["id","title","domain","published"]
    df_art = df_vec.groupby(group_cols, dropna=False).apply(aggregate_article_topics).reset_index()
    df_art["published_dt"] = pd.to_datetime(df_art["published"], errors="coerce")
    df_art = df_art.sort_values(by=["published_dt","domain","title"], ascending=[False, True, True]).reset_index(drop=True)

    print("\nAggregated per article:")
    display(df_art[["title","domain","published","topics_merged"]])


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


Per vector rows: 1675


/tmp/ipython-input-209050535.py:60: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_vec["published_dt"] = pd.to_datetime(df_vec["published"], errors="coerce")


,title,domain,published,topic_index,top_topic,topics_pretty
0,Idiotic NFL Fans Brawl It Out To Absolutely Pe...,dailycaller.com,2025-10-14T18:19:46+00:00,0,Sports / Global Athletics / Subdomain 1,Sports / Global Athletics / Subdomain 1 (0.79)...
1,Idiotic NFL Fans Brawl It Out To Absolutely Pe...,dailycaller.com,2025-10-14T18:19:46+00:00,1,Sports / Global Athletics / Subdomain 1,Sports / Global Athletics / Subdomain 1 (0.78)...
2,Busy Philipps calls out Dawsons Creek for ‘ins...,nypost.com,2025-10-14T14:15:34-04:00,0,Society / Culture & Identity / Subdomain 1,Society / Culture & Identity / Subdomain 1 (0....
3,Busy Philipps calls out Dawsons Creek for ‘ins...,nypost.com,2025-10-14T14:15:34-04:00,1,Society / Culture & Identity / Subdomain 1,Society / Culture & Identity / Subdomain 1 (0....
4,Busy Philipps calls out Dawsons Creek for ‘ins...,nypost.com,2025-10-14T14:15:34-04:00,2,Society / Culture & Identity / Subdomain 1,Society / Culture & Identity / Subdomain 1 (0....
...,...,...,...,...,...,...
1670,Why record harvests make famines far rarer — a...,www.vox.com,,1,Environment / Climate & Energy / Subdomain 1,Environment / Climate & Energy / Subdomain 1 (...
1671,Why record harvests make famines far rarer — a...,www.vox.com,,2,Society / Culture & Identity / Subdomain 1,Society / Culture & Identity / Subdomain 1 (0....
1672,Why record harvests make famines far rarer — a...,www.vox.com,,3,Politics / Global / Geopolitics / Subdomain 1,Politics / Global / Geopolitics / Subdomain 1 ...
1673,Why record harvests make famines far rarer — a...,www.vox.com,,4,Environment / Climate & Energy / Subdomain 1,Environment / Climate & Energy / Subdomain 1 (...



Aggregated per article:


/tmp/ipython-input-209050535.py:82: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_art = df_vec.groupby(group_cols, dropna=False).apply(aggregate_article_topics).reset_index()
/tmp/ipython-input-209050535.py:83: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_art["published_dt"] = pd.to_datetime(df_art["published"], errors="coerce")


,title,domain,published,topics_merged
0,Idiotic NFL Fans Brawl It Out To Absolutely Pe...,dailycaller.com,2025-10-14T18:19:46+00:00,Sports / Global Athletics / Subdomain 1 (0.79)...
1,Busy Philipps calls out Dawsons Creek for ‘ins...,nypost.com,2025-10-14T14:15:34-04:00,Society / Culture & Identity / Subdomain 1 (0....
2,House speaker defends White House using resear...,www.theguardian.com,2025-10-14T18:15:29+00:00,Politics / US / Federal / Subdomain 1 (0.84); ...
3,Trump Plays Democrats With Their Own Favorite ...,dailycaller.com,2025-10-14T18:15:17+00:00,Politics / US / Federal / Subdomain 1 (0.81); ...
4,Charlie Kirk honored with 'National Day of Rem...,nypost.com,2025-10-14T14:14:10-04:00,Society / Culture & Identity / Subdomain 1 (0....
...,...,...,...,...
429,Is it okay to like Harry Potter even though J....,www.vox.com,,Technology / Artificial Intelligence / Subdoma...
430,"Israel Hamas deal: The hostage, ceasefire, and...",www.vox.com,,Politics / Global / Geopolitics / Subdomain 1 ...
431,The backlash against Saudi Arabia’s Riyadh Com...,www.vox.com,,Society / Culture & Identity / Subdomain 1 (0....
432,The uncomfortable problem with America’s great...,www.vox.com,,Law / Justice & Civil Rights / Subdomain 1 (0....


# Setup 10 of N — Hybrid Dual-Model Stance Embeddings (FLAN-T5 + BART)

### **Purpose**
This step extracts each article’s *ideological stance vector* using a two-stage hybrid approach.  
The goal is to capture both the **structured ideological signal** (author worldview) and the **rhetorical expression** (how that worldview is communicated).

---

### **Pipeline Overview**

#### **1. Stage One — Ideological Classification (FLAN-T5)**
- Uses `google/flan-t5-large`, an instruction-tuned model capable of following precise prompts.
- The model receives a short portion of each article and is asked to produce exactly two lines:
  1. **Political leaning** — chosen from a fixed taxonomy (`progressive_left`, `centrist`, `libertarian`, `authoritarian_right`, etc.).
  2. **Implied stance** — the specific worldview or policy orientation (`climate_regulation_is_good`, `free_markets_are_best`, etc.).
- This gives a compact, discrete representation of the author’s ideological position.

#### **2. Stage Two — Rhetorical Summary (BART)**
- Uses `facebook/bart-large-cnn`, a high-quality abstractive summarizer.
- The model generates **one concise sentence** summarizing the article’s argument or tone.
- This stage is not asked to classify — only to write a short, natural-sounding summary.

#### **3. Stage Three — Hybrid Concatenation + Embedding**
- The two outputs are merged into a single text block:
- {political leaning}
-{implied stance}
-{one-sentence summary}


- This composite text is then embedded using the stance embedding model (`Snowflake/snowflake-arctic-embed-l`).
- The result is a **dense stance vector** that combines categorical ideological anchors with expressive natural-language context.

---

### **Why This Hybrid Works**
| Problem | Solution |
|----------|-----------|
| Models ignoring structured prompts | FLAN-T5 handles classification deterministically |
| Summaries too dry or inconsistent | BART provides fluent rhetorical phrasing |
| Labels getting lost in embeddings | They’re now always the first two lines of the text |
| Weak stance contrast in cosine space | Ideological anchors dominate the vector while summary refines nuance |

---

### **Outputs**
Each article will produce:
- A `label` variant (just the FLAN-T5 ideological pair)
- A `summary` variant (the full concatenated text for embedding)
- Both stored in the **stance Chroma collection** with metadata:
- `pov_label`
- `implied_stance`
- `stance_summary_text`
- `stance_variant` = `label` or `summary`

---

### **Result**
This setup creates high-fidelity, contrastive stance embeddings that:
- Preserve the author’s worldview structure
- Encode rhetorical tone and argument style
- Enable robust “opposite viewpoint” retrieval (same topic, opposing stance)

In short, it fuses the best parts of classification and summarization into one embedding space that can detect **what the article argues** and **how it argues it**.



In [11]:
# ==============================
# Setup 10 of N — Dual-Model Hybrid Stance Embeddings (FLAN-T5 + BART)
# ==============================
# Purpose:
#   1. Use FLAN-T5 to classify each article's political leaning & implied stance.
#   2. Use BART to produce one concise, fluent sentence summarizing its argument/tone.
#   3. Concatenate both outputs and embed using Snowflake Arctic embedder.
#   4. Upsert both label and summary variants into the stance Chroma collection.
# ==============================

import os, json, time, torch, numpy as np, logging
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import chromadb
import yaml

# --- Silence Chroma telemetry noise ---
logging.getLogger("chromadb.telemetry").setLevel(logging.CRITICAL)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running dual-model stance embedding pipeline on {device.upper()}")

# --- Paths and config ---
PROJECT_ROOT = Path("/content/anti_echo").resolve()
RAW_DIR = PROJECT_ROOT / "raw"
CONFIG_PATH = PROJECT_ROOT / "config/config.yaml"

CONFIG = yaml.safe_load(open(CONFIG_PATH, "r", encoding="utf-8"))

# --- Load ideological taxonomies ---
leanings_path = PROJECT_ROOT / "config/political_leanings.json"
stances_path  = PROJECT_ROOT / "config/implied_stances.json"

POLITICAL_LEANINGS = json.load(open(leanings_path, "r", encoding="utf-8"))
IMPLIED_STANCES    = json.load(open(stances_path, "r", encoding="utf-8"))

leaning_labels = list(POLITICAL_LEANINGS.keys())
stance_labels  = list(IMPLIED_STANCES.keys())

# --- Load models from config (v4 structure) ---
flan_model_name = CONFIG["stance_processing"]["classifier"]["model"]
bart_model_name = CONFIG["stance_processing"]["summarizer"]["model"]

print(f"Loading FLAN-T5 classifier: {flan_model_name}")
flan_tok = AutoTokenizer.from_pretrained(flan_model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_name).to(device)

print(f"Loading BART summarizer: {bart_model_name}")
bart_summarizer = pipeline(
    "summarization",
    model=bart_model_name,
    tokenizer=bart_model_name,
    device=0 if device == "cuda" else -1
)

# --- Embedding model (Snowflake Arctic) ---
stance_model_name = CONFIG["embeddings"]["stance_model"]
embedder = SentenceTransformer(stance_model_name, device=device)
stance_dtype = CONFIG["embeddings"]["dtype"]

# --- Chroma connection ---
client = chromadb.PersistentClient(path=str(PROJECT_ROOT / CONFIG["chroma"]["dir"]))
stance_coll = client.get_or_create_collection(CONFIG["chroma_collections"]["stance"])

# ==============================
# Helper functions
# ==============================

def flan_classify(text: str, title: str = "") -> tuple[str, str]:
    """Return (political_leaning, implied_stance) chosen ONLY from known label sets."""
    allowed_leanings = ", ".join(leaning_labels)
    allowed_stances = ", ".join(stance_labels)

    prompt = (
        "You are a classification model. "
        "Decide the author's implied political leaning and implied stance.\n"
        f"Political leaning (choose exactly one from this list): {allowed_leanings}\n"
        f"Implied stance (choose exactly one from this list): {allowed_stances}\n"
        "Respond with ONLY the two chosen labels, each on its own line, nothing else.\n\n"
        f"Title: {title}\n\nArticle:\n{text[:2000]}"
    )

    inputs = flan_tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    out_ids = flan_model.generate(**inputs, max_new_tokens=32, num_beams=4, early_stopping=True)
    output = flan_tok.decode(out_ids[0], skip_special_tokens=True)

    # --- Clean + normalize ---
    output = output.replace("Output:", "").replace("output:", "").replace("Answer:", "")
    lines = [ln.strip().lower() for ln in output.splitlines() if ln.strip()]

    leaning, stance = "", ""
    if len(lines) >= 2:
        leaning, stance = lines[0], lines[1]
    elif len(lines) == 1:
        leaning = lines[0]

    # --- Validate against allowed lists ---
    if leaning not in leaning_labels:
        leaning = "apolitical_or_unknown"
    if stance not in stance_labels:
        stance = "unknown"

    return leaning, stance


def bart_one_sentence(text: str) -> str:
    """Generate one-sentence rhetorical summary."""
    try:
        result = bart_summarizer(
            text[:3000],
            max_length=40,
            min_length=15,
            do_sample=False,
        )
        return result[0]["summary_text"].strip()
    except Exception:
        return ""


def sanitize(meta: dict):
    out = {}
    for k, v in meta.items():
        if isinstance(v, (str, int, float, bool)) or v is None:
            out[k] = "" if v is None else v
        else:
            out[k] = str(v)
    return out


def embed_texts(texts):
    vecs = embedder.encode(
        texts,
        normalize_embeddings=True,
        convert_to_numpy=True,
        show_progress_bar=False
    )
    dtype = np.float16 if stance_dtype == "float16" else np.float32
    return np.array(vecs, dtype=dtype)

# ==============================
# Main processing loop
# ==============================

start = time.time()
added, processed = 0, 0

for txt_path in RAW_DIR.glob("*.txt"):
    meta_path = txt_path.with_suffix(".meta.json")
    if not meta_path.exists():
        continue

    meta = json.load(open(meta_path, "r", encoding="utf-8"))
    text = open(txt_path, "r", encoding="utf-8").read().strip()
    title = meta.get("title", "")
    if not text:
        continue

    processed += 1
    try:
        # Stage 1: FLAN classification
        leaning, stance = flan_classify(text, title)

        # Stage 2: BART one-sentence summary
        summary = bart_one_sentence(text)

        # Stage 3: Concatenate for hybrid text
        hybrid_text = f"{leaning}\n{stance}\n{summary}".strip()
        label_text = f"{leaning}\n{stance}".strip()

        # --- Embeddings ---
        vec_label = embed_texts([label_text])[0]
        vec_hybrid = embed_texts([hybrid_text])[0]

        # --- Upsert label variant ---
        id_label = f"{meta['id']}::stance::label"
        stance_coll.upsert(
            ids=[id_label],
            embeddings=[vec_label.tolist()],
            metadatas=[sanitize({
                **meta,
                "stance_variant": "label",
                "political_leaning": leaning,
                "implied_stance": stance,
                "stance_summary_text": "",
            })]
        )

        # --- Upsert summary variant ---
        id_summary = f"{meta['id']}::stance::summary"
        stance_coll.upsert(
            ids=[id_summary],
            embeddings=[vec_hybrid.tolist()],
            metadatas=[sanitize({
                **meta,
                "stance_variant": "summary",
                "political_leaning": leaning,
                "implied_stance": stance,
                "stance_summary_text": summary,
            })]
        )

        added += 2
        if processed % 10 == 0:
            print(f"Processed {processed} articles...")

    except Exception as e:
        print(f"skip {meta.get('id','?')}: {type(e).__name__} {str(e)[:150]}")

elapsed = round(time.time() - start, 2)
print({"articles": processed, "rows_upserted": added, "elapsed_sec": elapsed})


Running dual-model stance embedding pipeline on CUDA
Loading FLAN-T5 classifier: google/flan-t5-large


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Processed 10 articles...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 20 articles...
Processed 30 articles...
Processed 40 articles...
Processed 50 articles...
Processed 60 articles...
Processed 70 articles...
Processed 80 articles...
Processed 90 articles...
Processed 100 articles...
Processed 110 articles...
Processed 120 articles...
Processed 130 articles...
Processed 140 articles...
Processed 150 articles...
Processed 160 articles...
Processed 170 articles...
Processed 180 articles...
Processed 190 articles...
Processed 200 articles...
Processed 210 articles...
Processed 220 articles...
Processed 230 articles...
Processed 240 articles...
Processed 250 articles...
Processed 260 articles...
Processed 270 articles...
Processed 280 articles...
Processed 290 articles...
Processed 300 articles...
Processed 310 articles...
Processed 320 articles...
Processed 330 articles...
Processed 340 articles...
Processed 350 articles...
Processed 360 articles...
Processed 370 articles...
Processed 380 articles...
Processed 390 articles...
Processed 400 articl

In [12]:
# ==============================
# Setup 10 Checker — Dual-Model Hybrid Output Inspector
# ==============================
import pandas as pd
import chromadb
from pathlib import Path

PROJECT_ROOT = Path("/content/anti_echo").resolve()
client = chromadb.PersistentClient(path=str(PROJECT_ROOT / CONFIG["chroma"]["dir"]))
stance_coll = client.get_collection(CONFIG["chroma_collections"]["stance"])

total = stance_coll.count()
print(f"Loaded Chroma collection '{CONFIG['chroma_collections']['stance']}' with {total} records")

# --- Pull metadata only ---
dump = stance_coll.get(include=["metadatas"], limit=total)

rows = []
for rid, meta in zip(dump["ids"], dump["metadatas"]):
    if not isinstance(meta, dict):
        continue
    rows.append({
        "row_id": rid,
        "stance_variant": meta.get("stance_variant", ""),
        "title": meta.get("title", ""),
        "domain": meta.get("domain", ""),
        "published": meta.get("published", ""),
        "political_leaning": meta.get("political_leaning", ""),
        "implied_stance": meta.get("implied_stance", ""),
        "stance_summary_text": meta.get("stance_summary_text", "")[:250]
    })

df = pd.DataFrame(rows)
if df.empty:
    raise ValueError("No stance records found in Chroma. Run Setup 10 first.")

# --- Normalize timestamps & sort ---
df["published_dt"] = pd.to_datetime(df["published"], errors="coerce", utc=True)
df = df.sort_values(by=["published_dt", "domain", "title"], ascending=[False, True, True]).reset_index(drop=True)

# --- Pivot: one row per article, columns for each stance variant ---
pivot = (
    df.pivot_table(
        index=["title", "domain", "published"],
        columns="stance_variant",
        values=["political_leaning", "implied_stance", "stance_summary_text"],
        aggfunc="first"
    )
    .reset_index()
    .fillna("")
)

# --- Flatten multi-index columns ---
pivot.columns = ['_'.join(col).strip("_") for col in pivot.columns.values]

# --- Build concise display table ---
pivot["label"] = (
    pivot["political_leaning_label"].astype(str) + "\n" +
    pivot["implied_stance_label"].astype(str)
)
pivot["summary"] = (
    pivot["political_leaning_summary"].astype(str) + "\n" +
    pivot["implied_stance_summary"].astype(str) + "\n" +
    pivot["stance_summary_text_summary"].astype(str)
)

# --- Final preview ---
cols = ["title", "domain", "published", "label", "summary"]
print(f"Articles: {len(pivot)} | Total stance entries: {len(df)}")
display(pivot[cols].head(25))


Loaded Chroma collection 'news_stance' with 868 records
Articles: 433 | Total stance entries: 868


,title,domain,published,label,summary
0,&#x27;The View&#x27; co-host breaks promise to...,www.foxnews.com,2025-10-14T13:30:10-04:00,apolitical_or_unknown\nunknown,"apolitical_or_unknown\nunknown\n""The View"" co-..."
1,&#x27;We want to raise our children&#x27;: Lif...,www.bbc.com,2025-10-14T16:30:50+00:00,authoritarian_left\nunknown,"authoritarian_left\nunknown\n""We are tired and..."
2,&quot;Viewpoint Diversity&quot; Requirements a...,reason.com,2025-10-13T13:04:21+00:00,apolitical_or_unknown\nunknown,apolitical_or_unknown\nunknown\nThe Trump Admi...
3,&quot;Viewpoint Diversity&quot; Requirements a...,reason.com,2025-10-14T12:01:41+00:00,center_left\nunknown,"center_left\nunknown\n""Viewpoint Diversity"" Re..."
4,&quot;Viewpoint Diversity&quot; Requirements a...,reason.com,2025-10-13T16:46:01+00:00,center_left\nunknown,"center_left\nunknown\n""Viewpoint Diversity"" Re..."
5,&quot;Viewpoint Diversity&quot; Requirements a...,reason.com,2025-10-14T16:53:59+00:00,authoritarian_left\nunknown,"authoritarian_left\nunknown\n""Viewpoint Divers..."
6,"1 gene, 1 disease no more – acknowledging the ...",theconversation.com,,apolitical_or_unknown\nunknown,apolitical_or_unknown\nunknown\nGenetic inheri...
7,24 hours with Trump on diplomatic tornado thro...,www.bbc.com,2025-10-14T15:08:41+00:00,authoritarian_right\nunknown,authoritarian_right\nunknown\nUS President Don...
8,250 Ballots Found In Amazon Box As ME Dems Fig...,thefederalist.com,2025-10-14T11:33:25+00:00,apolitical_or_unknown\nunknown,apolitical_or_unknown\nunknown\n250 ballots we...
9,"3-legged lizards can thrive against all odds, ...",theconversation.com,,apolitical_or_unknown\nunknown,apolitical_or_unknown\nunknown\nLizard biologi...


Setup 8 of N - persist feed state to HF and GitHub

Purpose
Snapshot feeds/feeds_state.json and feeds/index.json to HF as timestamped copies and latest pointers, and commit the same to GitHub. This keeps HF as the single source of truth while providing Git history.

Why this matters
Future runs and collaborators can always restore state. The UI you build later can also read the latest pointers.

Outputs
Four files on HF and two files in GitHub updated, with a short summary.

In [13]:
# Setup 8 of N: persist feed state to HF and GitHub
# Uploads timestamped snapshots and maintains latest pointers on HF and GitHub.

import os, json, base64
from datetime import datetime, timezone
from pathlib import Path
from huggingface_hub import upload_file
import requests
from getpass import getpass

PROJECT_ROOT = Path("/content/anti_echo").resolve()
FEEDS_DIR = PROJECT_ROOT / "feeds"
STATE_PATH = FEEDS_DIR / "feeds_state.json"
INDEX_PATH = FEEDS_DIR / "index.json"

# Inputs
HF_DATASET_ID = os.environ["HF_DATASET_ID"]
HF_TOKEN = os.environ.get("HF_TOKEN","").strip() or getpass("Enter HF token: ")
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN","").strip() or getpass("Enter GitHub token: ")
REPO_OWNER = "AHMerrill"
REPO_NAME = "anti-echo-chamber"
BRANCH = "main"

# Prepare timestamped names plus stable latest names
ts = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
uploads = [
    (STATE_PATH, f"feeds/feeds_state_{ts}.json"),
    (INDEX_PATH, f"feeds/feed_index_{ts}.json"),
    (STATE_PATH, "feeds/feeds_state_latest.json"),
    (INDEX_PATH, "feeds/feed_index_latest.json"),
]

# Upload to HF dataset
print("Uploading feed state to HF...")
for local, remote in uploads:
    upload_file(
        path_or_fileobj=str(local),
        path_in_repo=remote,
        repo_id=HF_DATASET_ID,
        repo_type="dataset",
        token=HF_TOKEN
    )
print("HF upload complete")

# Helper to upsert files in GitHub repo via REST API
def gh_put(local_path: Path, repo_path: str, message: str):
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{repo_path}"
    headers = {"Authorization": f"Bearer {GITHUB_TOKEN}", "Accept": "application/vnd.github+json"}
    content = local_path.read_bytes()
    # Fetch existing sha to update file in place if it already exists
    r = requests.get(url, headers=headers, timeout=20)
    sha = r.json().get("sha") if r.status_code == 200 else None
    payload = {"message": message, "content": base64.b64encode(content).decode(), "branch": BRANCH}
    if sha:
        payload["sha"] = sha
    resp = requests.put(url, headers=headers, json=payload, timeout=30)
    if resp.status_code not in (200,201):
        raise RuntimeError(f"GitHub push failed for {repo_path}: {resp.status_code} {resp.text[:300]}")

print("Committing feed state to GitHub...")
commit_msg = f"Update feed state and index {ts}"
for local, repo_path in [
    (STATE_PATH, f"feeds/feeds_state_{ts}.json"),
    (INDEX_PATH, f"feeds/feed_index_{ts}.json"),
    (STATE_PATH, "feeds/feeds_state_latest.json"),
    (INDEX_PATH, "feeds/feed_index_latest.json"),
]:
    gh_put(local, repo_path, commit_msg)
print("GitHub commit complete")


Uploading feed state to HF...
HF upload complete
Committing feed state to GitHub...
GitHub commit complete


Setup 11 of N - package and push batch to HF, update registry on Git

Purpose
Package current Chroma collections into batch files, upload to HF under batches/{batch_id}/, then update artifacts/artifacts_registry.json in your GitHub repo. HF is the single source of truth. The registry keeps a chronological ledger for rebuilds.

Why this matters
Gives you versioned, reconstructable artifacts and a single source of truth for future runs and for a UI.

Outputs
topic_embeddings.npz, stance_embeddings.npz, metadata.jsonl, manifest.json uploaded to HF and registry updated on GitHub.

In [14]:
# Setup 11 of N: package and push batch, update registry

import os, json, time, uuid, warnings, logging, requests, base64
from datetime import datetime, timezone
from pathlib import Path
import numpy as np
from huggingface_hub import upload_file
import chromadb

logging.getLogger("chromadb").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=DeprecationWarning)

PROJECT_ROOT = Path("/content/anti_echo").resolve()
BATCH_DIR = PROJECT_ROOT / CONFIG["batch"]["base_dir"]
HF_DATASET_ID = CONFIG["hf_dataset_id"]
REPO_OWNER = "AHMerrill"
REPO_NAME = "anti-echo-chamber"
BRANCH = "main"

client = chromadb.PersistentClient(path=str(PROJECT_ROOT / CONFIG["chroma"]["dir"]))
topic_coll = client.get_collection(CONFIG["chroma_collections"]["topic"])
stance_coll = client.get_collection(CONFIG["chroma_collections"]["stance"])

timestamp = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
batch_id = f"batch_{timestamp}_{uuid.uuid4().hex[:8]}"
batch_path = BATCH_DIR / batch_id
batch_path.mkdir(parents=True, exist_ok=True)

# Export collections with ids so we can align precisely
topic_dump = topic_coll.get(include=["embeddings","metadatas"])
stance_dump = stance_coll.get(include=["embeddings","metadatas"])

topic_vecs = np.array(topic_dump["embeddings"], dtype=np.float16)
stance_vecs = np.array(stance_dump["embeddings"], dtype=np.float16)

# File paths
topic_npz   = batch_path / CONFIG["batch"]["topic_file"]          # e.g. topic_embeddings.npz
stance_npz  = batch_path / CONFIG["batch"]["stance_file"]         # e.g. stance_embeddings.npz
meta_topic  = batch_path / "metadata_topic.jsonl"
meta_stance = batch_path / "metadata_stance.jsonl"
manifest_path = batch_path / CONFIG["batch"]["manifest_name"]     # e.g. manifest.json

# Write matrices
np.savez_compressed(topic_npz, topic_vecs)
np.savez_compressed(stance_npz, stance_vecs)

# Write aligned metadata jsonl for each collection
def write_meta_jsonl(path, ids, metas):
    with path.open("w", encoding="utf-8") as f:
        for rid, m in zip(ids, metas):
            rec = dict(m) if isinstance(m, dict) else {}
            rec["row_id"] = rid                  # exact vector id in Chroma for this row
            rec.setdefault("id", rec.get("id",""))
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

write_meta_jsonl(meta_topic,  topic_dump["ids"],  topic_dump["metadatas"])
write_meta_jsonl(meta_stance, stance_dump["ids"], stance_dump["metadatas"])

# Manifest with schema_version and separate metadata files
manifest = {
    "schema_version": 2,
    "batch_id": batch_id,
    "created_at": timestamp,
    "models": CONFIG["embeddings"],
    "counts": {"topic": int(topic_vecs.shape[0]), "stance": int(stance_vecs.shape[0])},
    "hf_dataset_id": HF_DATASET_ID,
    "paths": {
        "embeddings_topic": f"batches/{batch_id}/{topic_npz.name}",
        "embeddings_stance": f"batches/{batch_id}/{stance_npz.name}",
        "metadata_topic": f"batches/{batch_id}/{meta_topic.name}",
        "metadata_stance": f"batches/{batch_id}/{meta_stance.name}",
        "manifest": f"batches/{batch_id}/{manifest_path.name}",
    }
}
manifest_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")
print("Manifest:", json.dumps(manifest, indent=2))

# Upload artifacts to HF dataset
print("Uploading batch to HF...")
for fpath in [topic_npz, stance_npz, meta_topic, meta_stance, manifest_path]:
    upload_file(
        path_or_fileobj=str(fpath),
        path_in_repo=f"batches/{batch_id}/{fpath.name}",
        repo_id=HF_DATASET_ID,
        repo_type="dataset",
        token=os.environ["HF_TOKEN"]
    )
print("HF batch upload complete")

# Update registry on GitHub
REGISTRY_URL = f"https://raw.githubusercontent.com/{REPO_OWNER}/{REPO_NAME}/{BRANCH}/artifacts/artifacts_registry.json"
try:
    registry = requests.get(REGISTRY_URL, timeout=20).json()
except Exception:
    registry = {"version": 1, "models": {}, "batches": []}

registry.setdefault("batches", []).append(manifest)
if isinstance(registry.get("version"), int):
    registry["version"] += 1
else:
    registry["version"] = 1

new_registry_bytes = json.dumps(registry, indent=2).encode("utf-8")
url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/artifacts/artifacts_registry.json"
headers = {"Authorization": f"Bearer {os.environ['GITHUB_TOKEN']}", "Accept": "application/vnd.github+json"}
r = requests.get(url, headers=headers, timeout=20)
sha = r.json().get("sha") if r.status_code == 200 else None
payload = {
    "message": f"Update artifacts registry {timestamp}",
    "content": base64.b64encode(new_registry_bytes).decode(),
    "branch": BRANCH
}
if sha:
    payload["sha"] = sha
resp = requests.put(url, headers=headers, json=payload, timeout=30)
if resp.status_code not in (200,201):
    raise RuntimeError(f"GitHub registry push failed: {resp.status_code} {resp.text[:300]}")
print("Registry updated on GitHub")


Manifest: {
  "schema_version": 2,
  "batch_id": "batch_20251014T190038Z_9cc10549",
  "created_at": "20251014T190038Z",
  "models": {
    "topic_model": "intfloat/e5-base-v2",
    "stance_model": "Snowflake/snowflake-arctic-embed-l",
    "dim": 1024,
    "dtype": "float16",
    "pooling": "mean",
    "chunk_tokens": 512,
    "normalize": true
  },
  "counts": {
    "topic": 1675,
    "stance": 868
  },
  "hf_dataset_id": "zanimal/anti-echo-artifacts",
  "paths": {
    "embeddings_topic": "batches/batch_20251014T190038Z_9cc10549/embeddings_topic.npz",
    "embeddings_stance": "batches/batch_20251014T190038Z_9cc10549/embeddings_stance.npz",
    "metadata_topic": "batches/batch_20251014T190038Z_9cc10549/metadata_topic.jsonl",
    "metadata_stance": "batches/batch_20251014T190038Z_9cc10549/metadata_stance.jsonl",
    "manifest": "batches/batch_20251014T190038Z_9cc10549/manifest.json"
  }
}
Uploading batch to HF...


embeddings_topic.npz:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

embeddings_stance.npz:   0%|          | 0.00/936k [00:00<?, ?B/s]

HF batch upload complete
Registry updated on GitHub


Setup 12 of N - quick sanity query

Purpose
Verify both Chroma collections are populated and aligned. Useful when wiring a future UI that compares similar topics but dissimilar stances.

Why this matters
Catches empty collections or mismatched counts early.

Outputs
Counts for both collections and the collection names.

In [15]:
# Setup 12 of N: quick sanity query

import chromadb
from pathlib import Path

PROJECT_ROOT = Path("/content/anti_echo").resolve()
client = chromadb.PersistentClient(path=str(PROJECT_ROOT / CONFIG["chroma"]["dir"]))
topic_coll = client.get_collection(CONFIG["chroma_collections"]["topic"])
stance_coll = client.get_collection(CONFIG["chroma_collections"]["stance"])

print({
    "topic_count": topic_coll.count(),
    "stance_count": stance_coll.count(),
    "collections": CONFIG["chroma_collections"]
})


{'topic_count': 1675, 'stance_count': 868, 'collections': {'topic': 'news_topic', 'stance': 'news_stance'}}


Setup 13 of N - Verify topic labels were added

In [28]:
# ====================================================
# Setup 13 of N: Unified Chroma Metadata Inspector (final fixed version)
# Shows both topics + stances for same sampled articles
# ====================================================

import json, random
from pathlib import Path
import chromadb

PROJECT_ROOT = Path("/content/anti_echo").resolve()
client = chromadb.PersistentClient(path=str(PROJECT_ROOT / CONFIG["chroma"]["dir"]))
topic_coll = client.get_collection(CONFIG["chroma_collections"]["topic"])
stance_coll = client.get_collection(CONFIG["chroma_collections"]["stance"])

# --- Pull moderate sample and deduplicate by base article ID ---
topic_dump = topic_coll.get(include=["metadatas"], limit=200)
stance_dump = stance_coll.get(include=["metadatas"], limit=200)

topic_map, stance_map = {}, {}
for rid, meta in zip(topic_dump["ids"], topic_dump["metadatas"]):
    if not isinstance(meta, dict):
        continue
    art_id = rid.split("::")[0]
    topic_map.setdefault(art_id, []).append(meta)

for rid, meta in zip(stance_dump["ids"], stance_dump["metadatas"]):
    if not isinstance(meta, dict):
        continue
    art_id = rid.split("::")[0]
    stance_map.setdefault(art_id, []).append(meta)

# --- Union of all article IDs with either topic or stance ---
all_ids = sorted(set(topic_map.keys()) | set(stance_map.keys()))
sample_ids = random.sample(all_ids, min(5, len(all_ids)))

print(f"\n=== Inspecting {len(sample_ids)} sample articles across topic + stance ===")

for art_id in sample_ids:
    m_t = topic_map.get(art_id, [])
    m_s = stance_map.get(art_id, [])

    t0 = m_t[0] if m_t else {}
    s0 = m_s[0] if m_s else {}

    print("\n───────────────────────────────────────────────")
    print(f"Article ID: {art_id}")
    print(f"Title:  {t0.get('title') or s0.get('title') or '(untitled)'}")
    print(f"Domain: {t0.get('domain') or s0.get('domain') or 'unknown'}")
    print(f"Published: {t0.get('published') or s0.get('published') or 'unknown'}")

    # ----- Topics -----
    print("\n[Topics]")

    # Collect all topic entries for this article (across all topic_index entries)
    topics_all = []
    for mt in m_t:
        raw = mt.get("topic_labels_json") or mt.get("topics_flat") or ""
        if not raw:
            continue
        try:
            parsed = json.loads(raw) if isinstance(raw, str) else raw
            if isinstance(parsed, list):
                topics_all.extend(parsed)
        except Exception:
            continue

    if topics_all:
        shown = []
        for t in topics_all[:5]:
            if isinstance(t, dict):
                print(f"  • {t.get('topic_label','?')} ({t.get('similarity',0):.2f})")
                shown.append(t.get('topic_label'))
        print(f"  → top_topic: {t0.get('top_topic','(missing)')}")
    else:
        print("  No topics found in any topic vectors for this article.")

    # ----- Stance -----
    print("\n[Stance]")
    print(f"  Political leaning: {s0.get('political_leaning','(missing)')}")
    print(f"  Implied stance:    {s0.get('implied_stance','(missing)')}")
    summary = s0.get("stance_summary_text", "")
    if summary:
        print(f"  Summary: {summary[:200]}{'...' if len(summary)>200 else ''}")
    else:
        print("  No stance summary text found.")



=== Inspecting 5 sample articles across topic + stance ===

───────────────────────────────────────────────
Article ID: www.msnbc.com-untitled-730adf22deef
Title:  Untitled
Domain: www.msnbc.com
Published: 2025-10-14T16:17:29+00:00

[Topics]
  • Politics / Global / Geopolitics / Subdomain 1 (0.85)
  • Politics / Global / Geopolitics / Subdomain 2 (0.85)
  • Politics / Global / Geopolitics / Subdomain 3 (0.85)
  • Politics / Global / Geopolitics / Subdomain 4 (0.85)
  • Politics / Global / Geopolitics / Subdomain 5 (0.85)
  → top_topic: Politics / Global / Geopolitics / Subdomain 1

[Stance]
  Political leaning: apolitical_or_unknown
  Implied stance:    unknown
  No stance summary text found.

───────────────────────────────────────────────
Article ID: www.bbc.com-x27-we-want-to-raise-our-children-x27-life-in-gaza-after-cea-3307737923a5
Title:  &#x27;We want to raise our children&#x27;: Life in Gaza after ceasefire deal
Domain: www.bbc.com
Published: 2025-10-14T16:30:50+00:00

[Topics